In [1]:
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from itertools import combinations
from collections import Counter

In [2]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [3]:
okt = Okt()

In [4]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [5]:
def extract_keywords(text, num_keywords=10):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    return [keyword for keyword, _ in top_keywords]

In [6]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [7]:
len(df)

3611424

In [8]:
df.columns

Index(['id', 'topic', 'text'], dtype='object')

In [9]:
df_s = df[df.topic == '사회']

In [14]:
del df

In [15]:
len(df_s)

1808896

In [16]:
documents = df_s.text

In [ ]:
doc_keywords = []

for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

 48%|████████████▉              | 866653/1808896 [68:56:32<255:05:41,  1.03it/s]

In [ ]:
len(doc_keywords)

In [ ]:
len(doc_keywords)

In [76]:
doc_keywords

[['가락시장', '사람', '수산시장', '새벽', '시장', '배추', '서민', '경제', '상인', '가명'],
 ['의원', '최소한', '망언', '국민', '인간', '우리', '품위', '대한', '국회의원', '라면'],
 ['시민', '경제', '일찍', '공원', '기념', '운동', '보상', '정치인', '대구', '국채'],
 ['시민', '대전', '촛불', '이명박', '언론', '악법', '새해', '촛불집회', '노래', '노조'],
 ['어민', '생산', '장흥', '양식', '김양', '무산', '잡태', '처음', '어장', '생산량'],
 ['언론', '대구', '방송', '재벌', '정부', '통해', '문화제', '파업', '정책', '촛불'],
 ['서울', '시골', '대학', '똥값', '골탑', '농가', '우보', '뚜벅뚜벅', '만수', '조금'],
 ['국립공원', '사무소', '통일', '행사', '지리산', '경찰', '띄우기', '자제', '새해', '요청'],
 ['대표', '우리', '한해', '돌밭', '맹세', '나라', '옥답', '박희태', '역사', '이란'],
 ['지수', '사회', '공존', '사람', '지능지수', '능력', '위해', '다른', '인맥', '모두']]

In [59]:
cpu_count() / 2

48.0

In [60]:
len(doc_keywords)

1190

In [69]:
keyword_lists = parallel_keyword_extraction(documents[:10], num_keywords=10)

  0%|                                                    | 0/10 [00:00<?, ?it/s]


AssertionError: phrase input should be string, not <class 'tuple'>